# 1. Describe the methods and variables in the class [DiscreteEnv](https://github.com/openai/gym/blob/a5a6ae6bc0a5cfc0ff1ce9be723d59593c165022/gym/envs/toy_text/discrete.py#L17) which is the parent class of the Taxi V3 class.

- Has the following members:
    - s: current state
    - lastaction: last action
    - action_space: action space
    - observation_space: state space
    - nS: number of states
    - nA: number of actions
    - P: transitions (*)
    - isd: initial state distribution (**)
    - (*) dictionary of lists, where
    - P[s][a] == [(probability, nextstate, reward, done), ...]
    - (**) list or array of length nS

- Has the following methods:
    - def \_\_init\_\_(self, nS, nA, P, isd): initialize members
    - def seed(self, seed=None): get the seed of random number generator
    - def reset(self): reset current state to a random one, and lastaction to None
    - def step(self, a): take action of a and update the current state, last action, and return (int(s), r, d, {"prob": p}) where r is the reward, and d is the done flag

# 2. Describe the methods and variables in the [Taxi V3](https://github.com/openai/gym/blob/a5a6ae6bc0a5cfc0ff1ce9be723d59593c165022/gym/envs/toy_text/taxi.py) class.

- Has the following variables:
    - desc: np array of map description
    - locs: location of R, G, B, Y
    - lastaction: last action

- Has the following methods:
    - def \_\_init\_\_(self): initialize desc, locs, and compute tranisions P
    - def encode(self, taxi_row, taxi_col, pass_loc, dest_idx): encode taxi locations, passenger location, destination location into a state
    - def decode(self, i): decode taxi locations, passenger location, destination location from a give state
    - def render(self, mode='human'): display the map with the state information

# 3. Describe the Taxi V3 environment, its actions, states, reward structure and the rationale behind such a reward structure.

- Description:
    - There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends.
- Observations:
    - There are 500 discrete states since there are 25 taxi positions, 5 possible locations of the passenger (including the case when the passenger is in the taxi), and 4 destination locations. 
- Passenger locations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
    - 4: in taxi
- Destinations:
    - 0: R(ed)
    - 1: G(reen)
    - 2: Y(ellow)
    - 3: B(lue)
- Actions:
    - There are 6 discrete deterministic actions:
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger
- Rewards:
    - There is a default per-step reward of -1, except for delivering the passenger, which is +20, or executing "pickup" and "drop-off" actions illegally, which is -10.
- Rendering:
    - blue: passenger
    - magenta: destination
    - yellow: empty taxi
    - green: full taxi
    - other letters (R, G, Y and B): locations for passengers and destinations
- state space is represented by:
    - (taxi_row, taxi_col, passenger_location, destination)

# Train an algorithm to achieve a 100-episode average reward with a 5th percentile of 7.2 or higher and a 95th percentile of 8.2 or higher on the last 1000 episodes.

In [43]:
import gym
import numpy as np
import pandas as pd
import random as rd
from tqdm import tqdm

taxi = gym.make('Taxi-v3')
taxi.reset()
taxi.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [44]:
# find all terminal states
taxi.encode(taxi.locs[2][0], taxi.locs[2][1], 2, 2)
terminal_states = []
for i in range(4):
    loc = taxi.locs[i]
    ts= taxi.encode(loc[0], loc[1], i, i)
    terminal_states.append(ts)

print(terminal_states)

[0, 85, 410, 475]


In [45]:
def greedy_policy_from_returns_tb(table):
    policy = {s:None for s in range(taxi.observation_space.n) }
    for state in range(taxi.observation_space.n):
        if state not in terminal_states:
            greedy_action = np.argmax(table[state])
            policy[state] = greedy_action
            
    return policy

def pretty_print_policy(policy, taxi, drop_off=False, render=True):
    if render:
      taxi.render()
    taxi_row, taxi_col, pass_idx, dest_idx = taxi.decode(taxi.s)
    if drop_off: pass_idx = 4
    direction_repr = {1:' 🡑 ', 2:' 🡒 ', 3:' 🡐 ', 0:' 🡓 ', None:' ⬤ ', 4:' O ', 5:' X '}
    for row in range(5):
        for col in range(5):
            state = taxi.encode(row, col, pass_idx, dest_idx)
            print(direction_repr[policy[state]],end='')
        print()

In [46]:
Q = np.zeros([taxi.observation_space.n, taxi.action_space.n]) # np array is faster than pd dataframe for training

n_episodes = 5000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.999
alpha = 0.618
gama = 0.6
rewards = np.zeros(n_episodes)

for i in tqdm(range(n_episodes)):
    s0 = taxi.reset()
    done = False
    episode_reward = 0

    if (i+1)%500 == 0:
        print(f'Iteration {i+1}')
        policy = greedy_policy_from_returns_tb(Q)
        pretty_print_policy(policy, taxi, drop_off=False, render=True)
        pretty_print_policy(policy, taxi, drop_off=True, render=True)
        #print(Q)

    while not done:
        if rd.uniform(0, 1) < epsilon:
            a0 = taxi.action_space.sample()
        else:
            a0 = np.argmax(Q[s0])
        s1, reward, done, _  = taxi.step(a0)
        Q[s0,a0] += alpha*(reward + gama*Q[s1].max() - Q[s0,a0])
        episode_reward += reward
        
        s0 = s1

    epsilon *= epsilon_decay
    epsilon = max(epsilon,min_epsilon)
    
    rewards[i] = episode_reward

A_dct = {0:'South', 1:'North', 2:'East', 3:'West', 4:'Pickup', 5:'Dropoff'}
Q_table = pd.DataFrame(Q, columns = [A_dct[v] for v in A_dct]) # for better visualization of Q table

 11%|█         | 557/5000 [00:01<00:10, 426.20it/s]

Iteration 500
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡐  🡐  🡓  🡒  🡒 
 🡓  🡓  🡓  🡓  🡐 
 🡓  🡐  🡐  🡐  🡓 
 🡓  🡑  🡐  🡓  🡓 
 O  🡒  🡒  🡓  🡓 
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡒  🡓  🡒  🡒  X 
 🡒  🡓  🡒  🡒  🡑 
 🡒  🡒  🡒  🡒  🡑 
 🡑  🡑  🡑  🡑  🡑 
 🡑  🡐  🡑  🡒  🡑 


 23%|██▎       | 1131/5000 [00:02<00:05, 733.33it/s]

Iteration 1000
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 O  🡐  🡓  🡑  🡓 
 🡑  🡐  🡓  🡓  🡓 
 🡑  🡑  🡐  🡐  🡐 
 🡑  🡑  🡐  🡑  🡑 
 🡑  🡑  🡑  🡑  🡐 
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡓  🡓  🡓  🡓  🡐 
 🡓  🡐  🡓  🡐  🡐 
 🡓  🡐  🡐  🡐  🡐 
 🡓  🡑  🡑  🡑  🡐 
 X  🡑  🡑  🡑  🡑 


 33%|███▎      | 1639/5000 [00:03<00:03, 996.87it/s] 

...

Iteration 4000
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡒  🡓  🡒  🡓  🡓 
 🡒  🡓  🡓  🡓  🡓 
 🡒  🡒  🡒  🡓  🡓 
 🡑  🡒  🡑  🡓  🡐 
 🡑  🡒  🡑  O  🡐 
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 X  🡐  🡓  🡐  🡐 
 🡑  🡐  🡓  🡐  🡓 
 🡑  🡐  🡐  🡐  🡐 
 🡑  🡑  🡑  🡑  🡐 
 🡑  🡑  🡑  🡑  🡐 


 94%|█████████▎| 4681/5000 [00:05<00:00, 1379.14it/s]

Iteration 4500
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 O  🡐  🡓  🡐  🡓 
 🡑  🡐  🡓  🡓  🡓 
 🡑  🡐  🡐  🡐  🡐 
 🡑  🡑  🡐  🡑  🡑 
 🡑  🡑  🡑  🡑  🡐 
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡒  🡓  🡒  🡒  X 
 🡒  🡓  🡒  🡒  🡑 
 🡒  🡒  🡑  🡑  🡑 
 🡑  🡑  🡑  🡑  🡑 
 🡑  🡒  🡑  🡒  🡑 


100%|██████████| 5000/5000 [00:05<00:00, 903.16it/s] 

Iteration 5000
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡓  🡓  🡒  🡒  O 
 🡓  🡓  🡑  🡑  🡑 
 🡒  🡒  🡒  🡑  🡑 
 🡑  🡑  🡑  🡒  🡑 
 🡑  🡑  🡑  🡑  🡑 
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

 🡒  🡓  🡓  🡓  🡐 
 🡒  🡓  🡒  🡓  🡐 
 🡒  🡒  🡒  🡓  🡐 
 🡑  🡑  🡑  🡓  🡓 
 🡑  🡒  🡑  X  🡐 


In [47]:
# Validation
n_validate = 1000
rewards_validate = np.zeros(n_validate)
n_steps = np.zeros(n_validate)
penalties = 0
for i in range(n_validate):
    s0 = taxi.reset()
    done = False
    episode_reward = 0

    while not done:
        a0 = np.argmax(Q[s0])
        s1, reward, done, _  = taxi.step(a0)
        episode_reward += reward
        n_steps[i] += 1
        if reward == -10:
            penalties += 1

        s0 = s1
    
    rewards_validate[i] = episode_reward

rewards_validate_100mean = np.convolve(rewards_validate, np.ones(100), 'valid')/100
reward_5th = np.percentile(rewards_validate_100mean, 5)
reward_95th = np.percentile(rewards_validate_100mean, 95)
print()
print('5 percentile of rewards:', reward_5th, ', greater than 7.2:', reward_5th>=7.2)
print('95 percentile of rewards:', reward_95th, ', greater than 8.2:', reward_95th>=8.2)


5 percentile of rewards: 7.48 , greater than 7.2: True
95 percentile of rewards: 8.3 , greater than 8.2: True


# The algorithm should be able to perform pick-ups and dropoffs with zero penalties over 1000 episodes.

In [48]:
print('Penalties for wrong pickups and dropoffs over 1000 episodes:', penalties)

Penalties for wrong pickups and dropoffs over 1000 episodes: 0


# Document your solution including all hyper parameters and how those hyperparameters were selected.

- epsilon, min_epsilon, epsilon_decay are for epsilon-greedy search
    - epsilon = 1
    - min_epsilon = 0.1 
        - is not 0 because it is desired to still exploare the environment with small probablility after
    - epsilon_decay = 0.999

- alpha is the learning rate, a smaller alpha requires larger number of training episodes
    - alpha = 0.618

- gama is the discount factor, cannot be too small as it will dampen the far-future rewards too much; at the same time, it must be smaller than 1 otherwise it will not optimize the number of steps well
    - gama = 0.6 

In [49]:
import math
print('After',math.ceil(math.log(min_epsilon)/math.log(epsilon_decay)),'iterations, espilon decays to min_epsilon')

After 2302 iterations, espilon decays to min_epsilon


In [50]:
n_step_5th = np.percentile(n_steps, 5)
n_step_50th = np.percentile(n_steps, 50)
n_step_95th = np.percentile(n_steps, 95)
print(n_step_5th, n_step_50th, n_step_95th)

9.0 13.0 17.0
